In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio
import sqlite3
import math
import os
import json
import urllib
import datetime as dt

from matplotlib import image
from flask import g, Flask, render_template, request, session, redirect, url_for
from werkzeug.wrappers import Request, Response
from flask import Flask, render_template, request, jsonify
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

from werkzeug.serving import run_simple

In [2]:
con = sqlite3.connect('rijksstudio.db')

In [3]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE IF NOT EXISTS Clicks
               (painting_id INTEGER, 
                object_name TEXT, 
                user_id INTEGER,
                x INTEGER, 
                y INTEGER,
                score FLOAT,
                source TEXT)''')

cur.execute("""CREATE TABLE IF NOT EXISTS Users
               (user_id INTEGER PRIMARY KEY,
                username TEXT,
                password TEXT)""")

cur.execute("""CREATE TABLE IF NOT EXISTS Objects
               (painting_id TEXT,
                object_name TEXT,
                confidence INTEGER,
                min_x INTEGER, 
                min_y INTEGER,
                max_x INTEGER, 
                max_y INTEGER,
                source TEXT,
                total_clicks INTEGER DEFAULT 0,
                flags INTEGER DEFAULT 0
                )""")

# Save the changes
con.commit()
con.close()

In [4]:
# Connect to the database and setup the app
app = Flask(__name__)
app.secret_key = b'_5#y2L"F4Q8z\n\xec]/'

def get_db():
    # Connect to the database
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect("rijksstudio.db")
    return db

@app.teardown_appcontext
def close_connection(exception):
    # Close the database
    db = getattr(g, '_database', None)
    if db is not None:
        db.close()

In [5]:
@app.route('/', methods=["GET", "POST"])
def index():
    # If logged in, show a painting
    if "user_id" in session:
        
        session["user_type"] = calculate_user_type(session["user_id"])

        if session["user_type"] == 1: # We give the user a google vision object
            objects = pd.DataFrame(json.load(open("objects_vision.json")))
            objects = objects.append(pd.DataFrame(json.load(open("objects_hugging.json"))))

            painting = np.random.choice(objects['painting_id'])           
            everything = list(set(objects[objects['painting_id']==painting]['object_name'].to_list()))
                        
            # df = objects[objects['painting_id']==painting]
            df = pd.DataFrame({"painting_id": [painting for i in range(len(everything))], 
                               "object_name" : everything})
        else: # We trust the user, so we give them an NLP object
            # Choose random painting, but make sure there are objects in it
            objects = json.load(open("objects_nlp.json"))
            
            painting = np.random.choice(os.listdir("static/images/paintings/"))
            everything = list(set(objects[painting.split(".")[0]]))
            
            df = pd.DataFrame({"painting_id": [painting for i in range(len(everything))], 
                               "object_name" : everything})
        
        # Set painting
        session['painting'] = f"static/images/paintings/{df['painting_id'].values[0]}"        
        
        # Set list of objects
        session['all_objects'] = list(df['object_name'].values)
                
        # Choose one of the objects to be found
        obj = np.random.choice(session['all_objects'])
        session["all_objects"].remove(obj) 
        session["object"] = obj
        
        session["count"] = 0
        session["score"] = 0
        
        # Read image for dimensions
        dims = Image.open(session['painting']).size
        
        # Scaling
        ratio = min(1280 / dims[0], 720 / dims[1])
        session["ratio"] = ratio
                
        # Show the image on an html page
        return render_template('test_jim.html', 
                               name = obj.lower(), 
                               total_objects = len(session["all_objects"]),
                               url = f'static/images/paintings/{df["painting_id"].values[0]}', 
                               width = dims[0] * ratio, 
                               height = dims[1] * ratio,
                               user_type = session["user_type"],
                               username = session["username"])
    
    else: # Else, redirect the user to the login page
        return redirect(url_for("login"))

In [6]:
@app.route("/register", methods=["GET", "POST"])
def register():
    msg = ''
    # If getting the page, show the html
    if request.method == "GET":
        return render_template("register.html")
    else: # if posting the page, store the data
        username = request.form.get("username")
        password = request.form.get("password")
        
        cur = get_db().cursor()   
        q = cur.execute(f"""INSERT INTO Users
                           VALUES (null, '{username}', '{password}')
                           """)
        # Commit changes
        get_db().commit()

        row = cur.execute(f"""SELECT user_id, password, username
                               FROM Users 
                               WHERE username = '{request.form.get('username')}'""").fetchone()
            
        session["user_id"] = row[0]
        session["username"] = row[2]
        return redirect(url_for("index"))

In [7]:
@app.route("/login", methods=["GET", "POST"])
def login():
    """Compare a username and password against the database and let the user log in if both are correct."""

    # forget any user_id
    msg = ''
    session.clear()
    cur = get_db().cursor()
    
    # If posting, check input and attempt login
    if request.method == "POST":
        if not request.form.get("username") or not request.form.get("password"):
            return "please fill in everything"
                
        # Query for the user logging in 
        row = cur.execute(f"""SELECT user_id, password, username 
                               FROM Users 
                               WHERE username = '{request.form.get('username')}'""").fetchone()
            
        # If user not found or password does not match, throw an error
        if not row or not request.form.get("password") == row[1]:
            msg = 'Incorrect username / password!'
            return render_template("login.html", msg = msg)

        # remember which user has logged in
        session["user_id"] = row[0]
        session["username"] = row[2]
        
        # redirect user to home page
        return redirect(url_for("index"))
    
    else: # If getting, display html
        return render_template("login.html", msg = msg)

In [8]:
@app.route("/logout")
def logout():
    """Log user out."""

    # forget any user_id
    session.clear()

    # redirect user to login form
    return redirect(url_for("login"))

In [9]:
@app.route("/calculate_score", methods=["POST"])
def calculate_score():
    # Get AJAX-data
    r = request.get_json()
    x = r["x"]
    y = r["y"]

    # Scale X and Y values
    x *= 1 / session["ratio"]
    y *= 1 / session["ratio"]
    
    time = r["time"]
                
    # Load in the image
    img = np.array(Image.open(session['painting']))
    dims = Image.open(session['painting']).size
    
    if session["user_type"] == 1:
        plot, box = create_plot_API(img, x, y)
    else:
        # Create the plot/borders of the image
        plot, box, points = create_plot_NLP(img, x, y)
    
        # If there are not enough clicks to draw a border, just give the points
        if len(points) < 3:
            if session["all_objects"]:
                obj = np.random.choice(session["all_objects"])
                session["object"] = obj
                session["all_objects"].remove(obj)
            else:
                obj = ""

            session["rem"] = len(session["all_objects"])
            final_score = 100 - 2.5 * time

            session["count"] += 1
            session["score"] += final_score

            update_clicks(x, y,  final_score, "NLP")

            return jsonify({"msg" : f"+{final_score}!", 
                            "url" : plot, 
                            "obj": obj.lower(), 
                            "total_score": session["score"],
                            "rem" : session["rem"]})
    
    # Update clicks
    cur = get_db().cursor()
    row = cur.execute(f"""SELECT total_clicks 
                          FROM Objects 
                          WHERE painting_id = '{session["painting"].split("/")[-1]}' 
                          AND object_name = '{session["object"]}'""").fetchone()
    
    counter_total = row[0] + 1
    
    cur.execute(f"""UPDATE Objects 
                    SET total_clicks = {counter_total} 
                    WHERE painting_id = '{session["painting"].split("/")[-1]}' 
                    AND object_name = '{session["object"]}'
                    """)
    get_db().commit()
       
    # If there is only one box, find the score accordingly
    if isinstance(box, tuple):
        result = determine_score(box, x, y, time, plot)
    else: # If there are multiple boxes, find the highest score (aka closest click)
        result = (0, None)

        for b in box:
            current = determine_score(b, x, y, time, plot)
            if current[0] >= result[0]:
                result = current
    
    # Increase total score in session
    session["score"] += result[0]
    
    # Select the next object
    if session["all_objects"]:
        obj = np.random.choice(session["all_objects"])
        session["object"] = obj
        session["all_objects"].remove(obj)
    else:
        obj = ""
    
    # Update counters
    session["rem"] = len(session["all_objects"])
    session["count"] += 1
    
    # Store click in database
    update_clicks(x, y, result[0], ("NLP" if session["user_type"] == 2 else "GOOGLE_VISION"))

    # Return information to html
    return jsonify({"msg": f"+{result[0]}!", 
                    "url" : result[1], 
                    "obj": obj.lower(), 
                    "total_score": session["score"],
                    "rem": session["rem"]})

In [10]:
@app.route("/profile", methods = ["GET"])
def profile():
    
    cur = get_db().cursor()
    
    r = cur.execute(f"""SELECT * 
                        FROM Clicks
                        WHERE user_id = '{session["user_id"]}'""").fetchall()
    
    total_attempts = len(r)
    avg_score = np.round(np.array([i[5] for i in r]).mean(), 2)
    total_score = np.sum([i[5] for i in r])
    high_score = max([i[5] for i in r])
    unique_paintings = len(set([i[0] for i in r]))
    unique_objects = pd.DataFrame(r).groupby(0)[1].unique().str.len().sum()
    trust = ("entrusted" if session["user_type"] == 2 else "untrusted")
    
    return render_template("profile.html", 
                           unique = unique_paintings,
                           objects = unique_objects,
                           trust = trust,
                           atts = total_attempts, 
                           avg_score = avg_score, 
                           total_score = total_score,
                           high_score = high_score,
                           user_type = session["user_type"],
                           username = session["username"])

In [11]:
@app.route("/no_object", methods = ["POST"])
def no_object():
        
    painting = session["painting"].split("/")[-1]
    obj = session["object"]    
    img = np.array(Image.open(session['painting']))
    
    if session["user_type"] == 1:
        plot, box = create_plot_API(img, None, None)
    else:
        plot, box, points = create_plot_NLP(img, None, None, flagged = True)
        
    old = obj
        
    if session["all_objects"]:
        obj = np.random.choice(session["all_objects"])
        session["object"] = obj
        session["all_objects"].remove(obj)
    else:
        obj = ""
        
    session["rem"] = len(session["all_objects"])

    return jsonify({"msg1": "<br>Object reported!",
                    "msg2":
                    f"<br><br>Check the painting to see where our system thought the <u>{old.lower()}</u> was.<br>",
                    "url" : plot, 
                    "obj": obj.lower(),
                    "total_score": session["score"],
                    "rem": session["rem"]})

In [12]:
def update_clicks(x, y, score, source):
    # Store click in database
    cur = get_db().cursor()
    
    cur.execute(f"""INSERT INTO Clicks
                    VALUES ("{session["painting"].split("/")[-1]}", 
                            "{session["object"]}", 
                            "{session["user_id"]}", 
                            {x}, 
                            {y},
                            {score},
                            '{source}')""")
    
    get_db().commit()

In [13]:
def calculate_user_type(user_id):
    cur = get_db().cursor()   
    q = cur.execute(f"""SELECT score
                        FROM Clicks 
                        WHERE user_id == {user_id}""")   
           
    # Create border edges
    points = np.array([r[0] for r in q])
                      
    # If the user scores 50 points or higher on average, they are reliable
    if len(points) >= 10 and points.mean() >= 50:
        return 2
    else:
        return 1

In [14]:
def find_edges(points):
    """Find the edges of the user clicks to determine the boxes"""
    min_x = min(points, key = lambda x: x[0])[0]
    min_y = min(points, key = lambda x: x[1])[1]

    max_x = max(points, key = lambda x: x[0])[0]
    max_y = max(points, key = lambda x: x[1])[1]
    
    return min_x, min_y, max_x, max_y

In [15]:
def determine_clusters(points):
    # Convert to array
    a = np.array(points)
    
    # Assume no. clusters is 1 by default
    final_n = 1
    max_silhouette = 0.55  
        
    for n in range(2, 6):
        # Fit kmeans and calculate silhouette score
        kmeans = KMeans(n_clusters= n)    
        labels = kmeans.fit_predict(a)
        silhouette_avg = silhouette_score(a, labels)
                
        # If this silhouette score is better than all others
        # set the current no. clusters to be the result
        if silhouette_avg > max_silhouette:
            max_silhouette = silhouette_avg
            final_n = n
    
    # Using the best found n, cluster all data
    kmeans = KMeans(n_clusters = final_n)
    df = pd.DataFrame(a, columns = ["X", "Y"])
    df["label"] = kmeans.fit_predict(df)
    
    return df, final_n

In [16]:
def create_box(points):
    min_x, min_y, max_x, max_y = find_edges(points)
    box = (min_x, max_x, min_y, max_y)    
    return box

In [17]:
def shaver(points):
    """Shaves outliers off an array"""
    
    df = pd.DataFrame(points)
    to_shave = int(len(df) * 0.05)
        
    if to_shave >= 2:
        df = df.sort_values(0)[to_shave // 2:-to_shave // 2]
        df = df.sort_values(1)[to_shave // 2:-to_shave // 2]
    
    return df.sort_index().values

In [18]:
def create_plot_API(img, x, y):
    
        # Find boxes of current object
        cur = get_db().cursor()        
        boxes = cur.execute(f"""SELECT min_x, max_x, min_y, max_y
                                FROM Objects
                                WHERE painting_id = '{session['painting'].split("/")[-1]}'
                                AND UPPER(object_name) = '{session['object'].upper()}' 
                                AND (source='GOOGLE_VISION'
                                OR source = 'HUGGING_FACE')""").fetchall()
        
        # plot image
        plt.axis('off')
        plt.imshow(img)
        
        # If there is only one of the object in the painting, simply plot it
        if len(boxes) == 1:
            box = boxes[0]
            min_x, max_x, min_y, max_y = boxes[0]
            
            v_lines = [min_x, max_x]
            h_lines = [min_y, max_y]

            # Plot borders
            for line in v_lines:
                plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                           ls='-', lw=0.5)

            for line in h_lines:
                plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                           ls='-', lw=0.5)
            
        else: # Otherwise, plot all the borders
            for box in boxes:
                min_x, max_x, min_y, max_y = box
            
                v_lines = [min_x, max_x]
                h_lines = [min_y, max_y]

                # Plot borders
                for line in v_lines:
                    plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                               ls='-', lw=0.5)

                for line in h_lines:
                    plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                               ls='-', lw=0.5)
                    
            box = boxes
                
        # Store painting locally
        ob = session["object"]
        paint = session["painting"].split("/")[-1].replace(".png", "")
        loc = f'static/images/{paint}_{ob}_{dt.datetime.today().timestamp()}.png'

        if x and y:
            # Plot where the user clicked
            plt.scatter(x, y, c="r", s=0.5)

        # Store the rendered figure    
        plt.savefig(loc,
                    bbox_inches='tight',
                    dpi = 250, # set to like 500 if we want very pretty high quality paintings that take forever to load
                    pad_inches = 0) 

        plt.close()

        return loc, box

In [19]:
# turns a combination of painting and object name to an id
def create_obj_id(painting,obj_name):
    obj_id = ''.join([str(ord(char)-96) for char in obj_name])    # convert chars to numbers
    return (painting.split(".")[0] + obj_id) + "N"    # + G for Google Vision

In [20]:
def create_plot_NLP(img, x, y, flagged = False):   
    
    # Query the database for relevant clicks
    cur = get_db().cursor()   
    q = cur.execute(f"""SELECT x, y
                        FROM Clicks 
                        WHERE painting_id == '{session["painting"].split("/")[-1]}'
                        AND object_name == '{session["object"]}' 
                        AND source='NLP'""")
               
    # Create border edges
    points = [(r[0], r[1]) for r in q]    

    # plot image
    plt.axis('off')
    plt.imshow(img)
    
    painting = session["painting"].split("/")[-1]
    obj = session["object"]    
        
    # If there are at least three clicks, we can plot borders
    if len(points) > 2: 
        # If we only have a few clicks, do not make clusters yet
        points = shaver(points)
        
        if len(points) < 10:
            
            box = create_box(points)            
            min_x, max_x, min_y, max_y = box
            v_lines = [min_x, max_x]
            h_lines = [min_y, max_y]
            
            # Plot border
            for line in v_lines:
                plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                           ls='-', lw=0.5)

            for line in h_lines:
                plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                           ls='-', lw=0.5)
                
                
            update_objects(painting, obj, min_x, min_y, max_x, max_y, points, flagged)

            plt.scatter([i[0] for i in points], [i[1] for i in points], c="b", s=0.25)
        else:
            df, n_clusters = determine_clusters(points)            
            boxes = []
            
            for i in range(n_clusters):
                points = df[df["label"] == i][["X", "Y"]].values
                
                box = create_box(points)            
                boxes.append(box)
                min_x, max_x, min_y, max_y = box
                v_lines = [min_x, max_x]
                h_lines = [min_y, max_y]

                update_objects(painting, f"{obj}_{i}", min_x, min_y, max_x, max_y, points, flagged)
                
                # Plot borders
                for line in v_lines:
                    plt.vlines(x=line, color="r", ymin=min_y, ymax=max_y,
                               ls='-', lw=0.5)

                for line in h_lines:
                    plt.hlines(y=line, color="r", xmin=min_x, xmax=max_x,
                               ls='-', lw=0.5)
                    
                plt.scatter([i[0] for i in points], [i[1] for i in points], c="b", s=0.25)
                
            box = boxes
    else:
        box = None
        update_objects(painting, obj, 0, 0, 0, 0, points, flagged)

        
    get_db().commit()
    
    paint = session["painting"].split("/")[-1].replace(".png", "")
    loc = f'static/images/{paint}_{obj}_{dt.datetime.today().timestamp()}.png'
    
    if x and y:
        # Plot where the user clicked
        plt.scatter(x, y, c="r", s=0.5)
    
    # Store the rendered figure    
    plt.savefig(loc,
                bbox_inches='tight',
                dpi = 250, # set to like 500 if we want very pretty high quality paintings that take forever to load
                pad_inches = 0) 

    plt.close()
    
    return loc, box, points

In [21]:
def update_objects(painting, obj, min_x, min_y, max_x, max_y, points, flagged):
    
    cur = get_db().cursor()
    
    # Query for the current painting + object combination
    row = cur.execute(f"""SELECT total_clicks, flags 
                          FROM Objects 
                          WHERE painting_id = '{painting}' 
                          AND object_name = '{obj}'""").fetchone()

    # If we already have this in our database, update the scores
    if row:
        # Total clicks
        counter_total = row[0] + 1
        
        # Do we add a report yes or no
        if flagged:
            counter_flags = row[1] + 1
        else:
            counter_flags = row[1]

        # If we have more than 10 clicks, half of which are reports, remove the object
        if counter_flags / counter_total >= 0.5 and counter_total > 10:
            cur.execute(f"""DELETE FROM Objects
                            WHERE painting_id = '{painting}' 
                            AND object_name = '{obj}'""")

            get_db().commit()
            
            ## TODO: remove from NLP json file as well!
            
        else: # Otherwise, just add 1 to our clicks and reports
            cur.execute(f"""UPDATE Objects 
                        SET total_clicks = {counter_total}, 
                        flags = {counter_flags} 
                        WHERE painting_id = '{painting}'
                        AND object_name = '{obj}'
                        """)
            get_db().commit()
    else: # If this object is new, add it to the database
        cur.execute("""INSERT OR REPLACE INTO Objects 
                       VALUES (?,?,?,?,?,?,?,?,?,?);""", 
                       (painting, 
                        obj, 
                        0, 
                        min_x, 
                        min_y, 
                        max_x, 
                        max_y, 
                        "NLP",
                        len(points),
                        int(flagged)))

        get_db().commit()

In [22]:
def determine_score(box, x, y, time, plot):
    # max score is 100
    score = 100
    
    # score goes down by 2.5 per second
    score -= time * 2.5
    
    # extract box edges
    min_x, max_x, min_y, max_y = box

    # find the biggest box size
    size = max(max_x - min_x, max_y - min_y)
    
    # If the click is within the box, give full points
    if (min_x < x < max_x) and (min_y < y < max_y):
        
        return score, plot
    
    else:
        # distance to closest point on x-axis
        dx = max(min_x - x, 0, x - max_x)
        # distance to closest point on y-axis
        dy = max(min_y - y, 0, y - max_y)
        # pythagoras theorem
        distance = np.sqrt(dx**2 + dy**2)     
        
        if distance > size:
            final_score = 0
        else:
            final_score = min(score, np.round(math.exp(-4 * (distance / size)) * score, 2))
        
        return final_score, plot

In [23]:
def get_objects(source):
    cur = get_db().cursor()   
    
    paintings = cur.execute(f"""SELECT DISTINCT painting_id 
                                FROM Objects 
                                WHERE source='{source}'""").fetchall()
    painting = paintings[np.random.randint(0, len(paintings)-1)][0] # select random painting

    # get all objects from db
    q = cur.execute(f"""SELECT * 
                        FROM Objects 
                        WHERE source='{source}' 
                        AND painting_id='{painting}'""")
    
    cols = [column[0] for column in q.description]
    df = pd.DataFrame.from_records(data = q.fetchall(), columns = cols) 
    
    return df

In [24]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
